看上了一个公号的文章，想保留原格式批量下载成PDF文件，以便在本地阅读。如果想看老一点的文章，就再也不用去使劲往下滑了。要爬取文章，首先要获得文章的链接。公号不像网页，可直接“审查元素”获得链接，而需要通过抓包软件获取到相关参数。此处用到抓包软件`Fiddler`(可公众号回复“源文件”下载)及电脑版微信。
<br/>
<br/>安装好`Fiddler`后，会提示更新。其实就用这个版本就够了，不用浪费时间升级。打开界面，设置一下过滤，只抓取微信的包（如下图）。选中"Filters"，然后勾选"Use Filters"，再勾选"Show only if URL contains"，在后面的文本框中输入“mp.weixin”即可。

![](images\filter.png)

然后打开电脑版微信，进入目标公号，点击“查看历史消息”进入，然后鼠标往下滚动让公众号加载更多文章。这时就能在`Fiddler`界面看到抓取到的信息了。

![](images\wechat.png)

在界面左边选择“Inspectors”、“WebForms”和“JSON”（如下图），就能看到我们需要的信息。其中“WebForms”下面的“QueryString”里面就是请求信息，其中的一些关键参数在我们批量请求的时候要用到。“JSON”下面的“general_msg_list”里面就包含当前页面的所有文章的链接、标题、发表时间等信息。

![](images\get_weixin.png)
![](images\paras.png)

请求信息的参数中，除了`offset`会随着翻页而变化外，其他参数在每一页中都是固定不变的。`offset`是以10的倍数增加，即表示一页包含10篇文章，首页的`offset`值为0。另外，`key`会随着时间的变化而变化，如果突然不能获取到信息的话，很有可能就是`key`变了，需要重新抓包获取。
<br/>
<br/>我们按如下程序先来构建一个公号首页文章的请求，测试一下能不能工作。先导入`requests`和`json`库，分别用于请求目标链接获取信息，以及将包含信息的大字符串转换成python能够识别的格式。然后定义公号的链接，设置`headers`假装是浏览器在访问。`proxies`用于设置代理IP，它的格式是一个字典，有`http`与`https`两种，在爬取不同网站时需选用不同的`proxise`，此处将两种类型均放进去，`requests`会自动按需选择。然后将其它关键参数都从`Fiddler`界面中复制过来，并放入字典`params`。
<br/>
<br/>传入相关参数，并通过`requests.get()`请求获取信息，存入变量`response`，通过`response.text`显示获取到的结果。

In [74]:
import requests
import json
url = "https://mp.weixin.qq.com/mp/profile_ext" #公号的链接
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36 "}
proxies = {
    'https': None,
    'http': None,
}
uin = 'ODQ5NDE0MTMw'
# Key会随时间变化，需要获取最新的key，不然抓取不到数据
key = '3aa16237e25e21dabcda4cd2ced4c12ffa2a3653c9c8c5e8a397118866c992ff8c540f35d683436a36bb7fa70ef2e204b8428e96e40fd51b388ceeefb554ea0db4d8cecc43d2b9e2c2620f6f029401aa'
# 重要参数
params = {
    'action': 'getmsg',
    '__biz': 'MzU4ODg3MzAwNw==', #每个公众号的不一样，从Fiddler复制过来
    'f': 'json',
    'offset': 0, #控制翻页
    'count': '10',
    'is_ok': '1',
    'scene': '124',
    'uin': uin, #每个公众号的不一样，从Fiddler复制过来
    'key': key, #每个公众号的不一样，从Fiddler复制过来
    'wxtoken': '',
    'x5': '0',
}

# 获取公众号页面信息
response = requests.get(url, headers=headers, params=params, proxies=proxies)
print(response.text)

{"ret":0,"errmsg":"ok","msg_count":10,"can_msg_continue":1,"general_msg_list":"{\"list\":[{\"comm_msg_info\":{\"id\":1000000078,\"type\":49,\"datetime\":1594366155,\"fakeid\":\"3588873007\",\"status\":2,\"content\":\"\"},\"app_msg_ext_info\":{\"title\":\"Python保留格式复制多个excel工作表到汇总表并生成目录(实例69)\",\"digest\":\"保留原格式复制excel表，就数它强~\",\"content\":\"\",\"fileid\":100000741,\"content_url\":\"http:\/\/mp.weixin.qq.com\/s?__biz=MzU4ODg3MzAwNw==&amp;mid=2247484393&amp;idx=1&amp;sn=bf00bf10cd550866cfd0e3b4ecdf3696&amp;chksm=fdd76fa9caa0e6bf065f7a1b2f5f48a98a1cd5979e09fb4713f56b20abc5283bd2c5f014fb6e&amp;scene=27#wechat_redirect\",\"source_url\":\"\",\"cover\":\"http:\/\/mmbiz.qpic.cn\/mmbiz_jpg\/h9IkZnic79uO6fVCEFbuPnW4SzZjl25KuBiaaLd1HfTugZOibUCBibcUHicArcPBsGOwIWsCoYqtCHwL5YlQhzWCWWA\/0?wx_fmt=jpeg\",\"subtype\":9,\"is_multi\":0,\"multi_app_msg_item_list\":[],\"author\":\"PythonOffice\",\"copyright_stat\":11,\"duration\":0,\"del_flag\":1,\"item_show_type\":0,\"audio_fileid\":0,\"play_url\":\"\",\

通过上图可以看到，首页的10篇文章的信息就被获取到了。但此时的信息是一个大字符串，python无法识别。所以需要使用`json.loads()`转换，转换后存入`response_dict`。

In [75]:
type(response.text)

str

In [76]:
response_dict = json.loads(response.text)#将字符串转换成python能识别的格式
print(response_dict)

{'ret': 0, 'errmsg': 'ok', 'msg_count': 10, 'can_msg_continue': 1, 'general_msg_list': '{"list":[{"comm_msg_info":{"id":1000000078,"type":49,"datetime":1594366155,"fakeid":"3588873007","status":2,"content":""},"app_msg_ext_info":{"title":"Python保留格式复制多个excel工作表到汇总表并生成目录(实例69)","digest":"保留原格式复制excel表，就数它强~","content":"","fileid":100000741,"content_url":"http://mp.weixin.qq.com/s?__biz=MzU4ODg3MzAwNw==&amp;mid=2247484393&amp;idx=1&amp;sn=bf00bf10cd550866cfd0e3b4ecdf3696&amp;chksm=fdd76fa9caa0e6bf065f7a1b2f5f48a98a1cd5979e09fb4713f56b20abc5283bd2c5f014fb6e&amp;scene=27#wechat_redirect","source_url":"","cover":"http://mmbiz.qpic.cn/mmbiz_jpg/h9IkZnic79uO6fVCEFbuPnW4SzZjl25KuBiaaLd1HfTugZOibUCBibcUHicArcPBsGOwIWsCoYqtCHwL5YlQhzWCWWA/0?wx_fmt=jpeg","subtype":9,"is_multi":0,"multi_app_msg_item_list":[],"author":"PythonOffice","copyright_stat":11,"duration":0,"del_flag":1,"item_show_type":0,"audio_fileid":0,"play_url":"","malicious_title_reason_id":0,"malicious_content_type":0}},{"comm_msg_in

`response_dict`是一个字典，其中两个关键的键是` 'can_msg_continue'`和`'general_msg_list'`，它们分别含有“后面页是否还有文章”及“当前页的10篇文章的信息”。如果` 'can_msg_continue'`的值是1，则表示后面页还有文章，如果是0，则没有了。我们可以通过判断它的值来决定何时终止程序，避免浪费时间。
<br/>
<br/>当前页的10篇文章的信息位于`general_msg_list`键里面`list`键对应的值里面。要提取信息，需要先获取`general_msg_list`的值，存入`general_msg_list`变量。此时`general_msg_list`是一个字符串，还是需要用`json.loads()`转换，转换后存入`data_list`，打印显示一下。振作精神，离目标越来越近啦！

In [77]:
general_msg_list = response_dict['general_msg_list'] #此时是字符串
data_list = json.loads(general_msg_list)['list'] #转换成python能够识别的格式，并取出文章链接的信息
print(data_list)

[{'comm_msg_info': {'id': 1000000078, 'type': 49, 'datetime': 1594366155, 'fakeid': '3588873007', 'status': 2, 'content': ''}, 'app_msg_ext_info': {'title': 'Python保留格式复制多个excel工作表到汇总表并生成目录(实例69)', 'digest': '保留原格式复制excel表，就数它强~', 'content': '', 'fileid': 100000741, 'content_url': 'http://mp.weixin.qq.com/s?__biz=MzU4ODg3MzAwNw==&amp;mid=2247484393&amp;idx=1&amp;sn=bf00bf10cd550866cfd0e3b4ecdf3696&amp;chksm=fdd76fa9caa0e6bf065f7a1b2f5f48a98a1cd5979e09fb4713f56b20abc5283bd2c5f014fb6e&amp;scene=27#wechat_redirect', 'source_url': '', 'cover': 'http://mmbiz.qpic.cn/mmbiz_jpg/h9IkZnic79uO6fVCEFbuPnW4SzZjl25KuBiaaLd1HfTugZOibUCBibcUHicArcPBsGOwIWsCoYqtCHwL5YlQhzWCWWA/0?wx_fmt=jpeg', 'subtype': 9, 'is_multi': 0, 'multi_app_msg_item_list': [], 'author': 'PythonOffice', 'copyright_stat': 11, 'duration': 0, 'del_flag': 1, 'item_show_type': 0, 'audio_fileid': 0, 'play_url': '', 'malicious_title_reason_id': 0, 'malicious_content_type': 0}}, {'comm_msg_info': {'id': 1000000077, 'type': 49, 'datetim

使用`data_list[0]`取第一个元素看一下，找到我们需要的信息，即时间（`datetime`），标题（`title`）和链接（`content_url`）。通过获取这些信息，就能为最后的下载做好准备了。

In [78]:
data_list[0]

{'comm_msg_info': {'id': 1000000078,
  'type': 49,
  'datetime': 1594366155,
  'fakeid': '3588873007',
  'status': 2,
  'content': ''},
 'app_msg_ext_info': {'title': 'Python保留格式复制多个excel工作表到汇总表并生成目录(实例69)',
  'digest': '保留原格式复制excel表，就数它强~',
  'content': '',
  'fileid': 100000741,
  'content_url': 'http://mp.weixin.qq.com/s?__biz=MzU4ODg3MzAwNw==&amp;mid=2247484393&amp;idx=1&amp;sn=bf00bf10cd550866cfd0e3b4ecdf3696&amp;chksm=fdd76fa9caa0e6bf065f7a1b2f5f48a98a1cd5979e09fb4713f56b20abc5283bd2c5f014fb6e&amp;scene=27#wechat_redirect',
  'source_url': '',
  'cover': 'http://mmbiz.qpic.cn/mmbiz_jpg/h9IkZnic79uO6fVCEFbuPnW4SzZjl25KuBiaaLd1HfTugZOibUCBibcUHicArcPBsGOwIWsCoYqtCHwL5YlQhzWCWWA/0?wx_fmt=jpeg',
  'subtype': 9,
  'is_multi': 0,
  'multi_app_msg_item_list': [],
  'author': 'PythonOffice',
  'copyright_stat': 11,
  'duration': 0,
  'del_flag': 1,
  'item_show_type': 0,
  'audio_fileid': 0,
  'play_url': '',
  'malicious_title_reason_id': 0,
  'malicious_content_type': 0}}

单页的内容搞定后，就可以获取所有文章链接的信息了。定义一个函数`article_links`来获取单页的信息，参数`index`用于控制页数。由于部分公众号文章是“消息”，因此`data`里的数据缺少我们需要的字段，需要跳过，所以加入了`try - except`语句，不然程序会崩溃。最后将单页的10篇文章信息存入列表`links`，并返回。同时返回的还有`can_msg_continue`，它用于判断当前页是否是最后一页。

In [87]:
import requests
import json
import time
def article_links(index):
    '''用于将单个页面的文章链接，index用于控制页数'''
    links = []
    url = "https://mp.weixin.qq.com/mp/profile_ext"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36 "}
    proxies = {'https': None,'http': None} #设定代理IP，包含'http'和'https'
    uin = 'ODQ5NDE0MTMw'
    # Key会随时间变化，需要获取最新的key，不然抓取不到数据
    key = '3aa16237e25e21dabcda4cd2ced4c12ffa2a3653c9c8c5e8a397118866c992ff8c540f35d683436a36bb7fa70ef2e204b8428e96e40fd51b388ceeefb554ea0db4d8cecc43d2b9e2c2620f6f029401aa'
    # 参数
    params = {
        'action': 'getmsg',
        '__biz': 'MzU4ODg3MzAwNw==', #每个公众号的不一样，从Fiddler复制过来
        'f': 'json',
        'offset': index * 10, #控制翻页，offset是从0开始的10的倍数
        'count': '10',
        'is_ok': '1',
        'scene': '124',
        'uin': uin, #每个公众号的不一样，从Fiddler复制过来
        'key': key, #每个公众号的不一样，从Fiddler复制过来
        'wxtoken': '',
        'x5': '0'}
    response = requests.get(url, headers=headers, params=params, proxies=proxies)
    response_dict = json.loads(response.text)
    can_msg_continue = response_dict.get('can_msg_continue') #用于判断是否是最后一页,若为0,则表示最后一页
    general_msg_list = response_dict.get('general_msg_list')
    data_list = json.loads(general_msg_list)['list']
    for data in data_list:
        #部分公众号文章是“消息”，因此data里的数据缺少我们需要的字段，需要跳过
        try:
            title = data['app_msg_ext_info']['title'] #文章标题
            datetime = data['comm_msg_info']['datetime'] #获取时间戳
            date = time.strftime('%Y-%m-%d', time.localtime(datetime))#将时间戳转换为本地时间
            url = data['app_msg_ext_info']['content_url'] #文章链接
            info = { "url":url,
                    "title":title,
                    "date":date}
            links.append(info)
        except:
            pass
    return links,can_msg_continue

然后就可以调用`article_links`函数来获取公号的所有文章链接信息了。在一个整数范围内用`for`循环遍历。此处设定为100，其中一个数对应的是1页，即10篇文章，那100就对应1000篇文章。这个要根据公号的实际文章数来设定，只要大于文章总数即可。然后将获取的链接信息通过`extend`存入列表`all_links`（如果用`append`会造成嵌套列表，会增加提取数据的难度）。再判断一下当前页是否是最后一页，如果是，就表示已经爬完所有文章信息了，通过`break`中断循环，避免浪费时间。最后查看一下前两篇文章的信息以及文章个数是否与公号总文章数吻合。

In [88]:
all_links = []
for i in range(100): #根据实际而定，因为每页10篇文章，100页对应1000篇文章
    all_links.extend(article_links(i)[0])
    if article_links(i)[1] == 0:
        break
all_links[:2]

[{'url': 'http://mp.weixin.qq.com/s?__biz=MzU4ODg3MzAwNw==&amp;mid=2247484393&amp;idx=1&amp;sn=bf00bf10cd550866cfd0e3b4ecdf3696&amp;chksm=fdd76fa9caa0e6bf065f7a1b2f5f48a98a1cd5979e09fb4713f56b20abc5283bd2c5f014fb6e&amp;scene=27#wechat_redirect',
  'title': 'Python保留格式复制多个excel工作表到汇总表并生成目录(实例69)',
  'date': '2020-07-10'},
 {'url': 'http://mp.weixin.qq.com/s?__biz=MzU4ODg3MzAwNw==&amp;mid=2247484385&amp;idx=1&amp;sn=30e05ced3e5a850eb867b257c7b2325a&amp;chksm=fdd76fa1caa0e6b7065945315ac2179bd8f4bc1a8171221938b1d84373f78ccb16144e801239&amp;scene=27#wechat_redirect',
  'title': 'Python批量新建文件夹并保存日志信息(实例68)',
  'date': '2020-07-09'}]

In [89]:
len(all_links)

74

链接获取完后，就可以批量下载成PDF文件了。此处用到`pdfkit`工具，需要自行通过`pip install pdfkit`安装。`pdfkit`只是一个`python`接口，其背后真正支持的程序是`wkhtmltopdf.exe`，所以需要安装`wkhtmltopdf.exe`。这个文件已上传网盘，可到公众号输入“源文件”下载。这个支持Windows XP/2003以后的系统。将`“wkhtmltox.rar”`压缩包解压，并记录其中的`wkhtmltopdf.exe`文件的路径，待会设置配置参数要用到。比如，本文将其放在`D:\Program Files (x86)`目录下。
![](images\wkhtmltox.PNG)

如果直接使用`pdfkit.from_url`转存PDF，会导致PDF中无法正常显示公号文章里的图片。因此还要用到`wechatsogou`库来处理一下。它是基于搜狗微信搜索的公号爬虫接口，也可以用它来爬公号文章。小编是怎么知道的呢？其实先前也不知道，遇到了这个问题，度娘出来的。反正能解决问题，使用别人造好的轮子，没毛病。于是乎，一顿凶猛的操作，`pip install wechatsogou`安装完毕，无耻地复制粘贴别人的代码。
<br/>
<br/>定义函数`link_to_pdf`，三个参数（`url,title,date`）都来自以上获取到的信息。然后调用`WechatSogouAPI`接口程序，请求链接，将获取到的信息转成`html`格式。然后将标题和文章内容放入写好的`html`中处理一下。指定好`wkhtmltopdf`可执行程序路径，配置一下`pdfkit`的参数，然后使用`pdfkit.from_string`方法，传入`html`，输出文件的文件名，以及配置信息，开始下载。

In [67]:
import pdfkit
import wechatsogou
def link_to_pdf(url,title,date):
    ws_api = wechatsogou.WechatSogouAPI(captcha_break_time=3) #调用接口
    content_info = ws_api.get_article_content(url) #请求链接
    content = content_info['content_html'] #转换为html格式
    #将标题和文章内容放入如下html中处理一下
    html = f'''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <title>{title}</title>
        </head>
        <body>
        <h2 style="text-align: center;font-weight: 400;">{title}</h2>
        {content}
        </body>
        </html>'''

    path_wkthmltopdf = r"D:\Program Files (x86)\wkhtmltox\bin\wkhtmltopdf.exe" #wkhtmltopdf可执行程序路径
    config = pdfkit.configuration(wkhtmltopdf = path_wkthmltopdf) #配置pdfkit
    pdfkit.from_string(html,f"{title} {date}.pdf",configuration=config) #转PDF，并按设定好的命名文件
    print(f"{title}.pdf 已下载")

在运行程序前，要把抓包软件`Fiddler`关闭，不然会报错“SSLError: HTTPSConnectionPool(host='mp.weixin.qq.com', port=443)”。使用`for`循环，遍历列表`all_links`，调用函数`link_to_pdf`开始工作。

In [66]:
for link in all_links:
    url = link['url']
    title = link['title']
    date = link['date']
    link_to_pdf(url, title, date)

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
Python保留格式复制多个excel工作表到汇总表并生成目录(实例69).pdf 已下载
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
Python批量新建文件夹并保存日志信息(实例68).pdf 已下载
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
P

KeyboardInterrupt: 

下载好的PDF如下图所示，还带目录的哦。然后再将所有单个的PDF合并成一个，就实现了公号文章汇总啦，是不是香喷喷啊？ PDF合并可参考实例4 （https://zhuanlan.zhihu.com/p/60685930） 。

![](images/result.png)